In [ ]:
import plotly.graph_objects as go
import ipywidgets as widgets
from plotlysources import getSourceTrace
import numpy as np
import magpylib as magpy
import plotly.figure_factory as ff
import plotly.io as pio
import glob
import os
import json

pio.templates['plotly_grey'] = pio.to_templated(go.Figure(layout_paper_bgcolor='rgb(33,33,33)', 
                                                          layout_plot_bgcolor='rgb(33,33,33)', 
                                                          layout_template='plotly_dark')).layout.template
default_theme = 'plotly'
debug_view = widgets.Output(layout={'border': '1px solid black'})

# Streamlines

In [ ]:
@debug_view.capture(clear_output=True)
def show_streamlines(change=None):
        sk = streamlines_options
        test=[]
        for k,v in sk.items():
            test.append(v['checkbox'].value and v['density'].value>0.5)
        if sum(test)>0:
            continuous_update_checkbox.value = False
        
        for k,v in streamlines_options.items():
            if v['checkbox'].value == False:
                v['position'].layout.visibility = 'hidden'
                v['density'].layout.visibility = 'hidden'
            else:
                v['position'].layout.visibility = 'visible'
                v['density'].layout.visibility = 'visible'
        
        try:

            sr = 100
            N=2
            if sk['xy']['checkbox'].value == True:
                pos = sk['xy']['position'].value
                density = sk['xy']['density'].value
                xs= np.linspace(-sr,sr,int(N*density))+0.01
                ys= np.linspace(-sr,sr,int(N*density))+0.01
                Bs = np.array([[pmc.getB([x,y,pos]) for x in xs] for y in ys])
                U,V = Bs[:,:,0], Bs[:,:,1]
                streamline = ff.create_streamline(x=xs, y=ys , u=U, v=V, density = 0.5*density, arrow_scale=sr*0.05)
                sl = streamline.data[0]
                t = figmag.data[1]
                t.mode = 'lines'
                with figmag.batch_update():
                    t.visible = True
                    t.x=sl.x ; t.y=sl.y ;  t.z=np.ones(len(sl.x))*pos
            else:
                figmag.data[1].visible = False

            if sk['xz']['checkbox'].value == True:
                pos = sk['xz']['position'].value
                density = sk['xz']['density'].value
                xs= np.linspace(-sr,sr,int(N*density))+0.01
                zs= np.linspace(-sr,sr,int(N*density))+0.01
                Bs = np.array([[pmc.getB([x,pos,z]) for x in xs] for z in zs])
                U,V = Bs[:,:,0], Bs[:,:,2]
                streamline = ff.create_streamline(x=xs, y=zs , u=U, v=V, density = 0.5*density, arrow_scale=sr*0.05)
                sl = streamline.data[0]
                t = figmag.data[2]
                t.mode = 'lines'
                with figmag.batch_update():
                    t.visible = True
                    t.x=sl.x ; t.y=np.ones(len(sl.x))*pos ;  t.z=sl.y
            else:
                figmag.data[2].visible = False

            if sk['yz']['checkbox'].value == True:
                pos = sk['yz']['position'].value
                density = sk['yz']['density'].value
                ys= np.linspace(-sr,sr,int(N*density))+0.01
                zs= np.linspace(-sr,sr,int(N*density))+0.01
                Bs = np.array([[pmc.getB([pos,y,z]) for y in ys] for z in zs])
                U,V = Bs[:,:,1], Bs[:,:,2]
                streamline = ff.create_streamline(x=ys, y=zs , u=U, v=V, density = 0.5*density, arrow_scale=sr*0.05)
                sl = streamline.data[0]
                t = figmag.data[3]
                t.mode = 'lines'
                with figmag.batch_update():
                    t.visible = True
                    t.x=np.ones(len(sl.x))*pos ; t.y=sl.x ;  t.z=sl.y
            else:
                figmag.data[3].visible = False
            
        except Exception as e:
            figmag.data[1].visible = False
            figmag.data[2].visible = False
            figmag.data[3].visible = False
            print(e)
        
        
def _define_streamlines_widgets(density=1):
    streamlines_options={}
    for k,p in zip(['xy', 'xz', 'yz'],['z', 'y', 'x']):
        sk = streamlines_options[k] = dict()
        sk['checkbox'] = widgets.Checkbox(description=f'{k}-plane',  layout=dict(width='auto'), style=dict(description_width='0px'))
        sk['position'] = widgets.FloatSlider(description=f'{p}-position', min=-scene_range, max=scene_range, value=0, step=0.1, 
                                             continuous_update=False, layout=dict(flex='1'))
        sk['density'] = widgets.BoundedFloatText(description='density', min=1, max=5, step = 1, value=density, 
                                                 continuous_update=False, layout=dict(width='auto'), style=dict(description_width='auto'))
        sk['container']= widgets.HBox([sk['checkbox'], sk['position'], sk['density']], layout=dict(justify_content='space-between'))

        for s in sk.values():
            s.tag = k
            s.observe(show_streamlines, names='value')

    return streamlines_options,  widgets.VBox([v['container'] for v in streamlines_options.values()])

# Isosurface

In [ ]:
@debug_view.capture(clear_output=True)
def _define_isosurface_widgets(sr=100, density=20, opacity=0.5, surface_count=20):
    '''sr -> isosurface range'''
    density=20
    sr = 100
    isosurface_widgets_dict = dict(
        density_x = widgets.BoundedIntText(description='x', min=5, max=50, value=density, layout=dict(width='60px')),
        density_y = widgets.BoundedIntText(description='y', min=5, max=50, value=density, layout=dict(width='60px')),
        density_z = widgets.BoundedIntText(description='z', min=5, max=50, value=density, layout=dict(width='60px')),
        srx = widgets.FloatRangeSlider(description='x-range [mm]', min=-sr, max=sr, value=(-sr,sr), layout=dict(width='auto')),
        sry = widgets.FloatRangeSlider(description='y-range [mm]', min=-sr, max=sr, value=(-sr,sr), layout=dict(width='auto')),
        srz = widgets.FloatRangeSlider(description='z-range [mm]', min=-sr, max=sr, value=(-sr,sr), layout=dict(width='auto')),
        opacity = widgets.FloatSlider(description='opacity', min=0, max=1, step=0.1, value=opacity, layout=dict(width='auto')),
        surface_count = widgets.BoundedIntText(description='surf count', min=1, max=50, value=surface_count, layout=dict(width='110px')),
        isominmax = widgets.FloatRangeSlider(description='iso [%]', min=0, max=100, value=(10,100), layout=dict(width='auto'))
    )
    for w in isosurface_widgets_dict.values():
        w.style.description_width='auto'

    @debug_view.capture(clear_output=True)
    def update_isosurface(srx=(-sr,sr), sry=(-sr,sr), srz=(-sr,sr), density_x=20, density_y=20, density_z=20, surface_count=10, opacity=0.5,
                         isominmax = (0,10)):
        for v in isosurface_widgets_dict.values():
            v.disabled = True
            v.layout.flex ='1'
        X, Y, Z = np.mgrid[srx[0]:srx[1]:density_x*1j, sry[0]:sry[1]:density_y*1j, srz[0]:srz[1]:density_z*1j]
        x = X.flatten()
        y = Y.flatten()
        z = Z.flatten()

        Bs = np.array([np.linalg.norm(pmc.getB([x,y,z])) for x,y,z in zip(x,y,z)]).flatten()
        imin = isominmax[0]*(np.max(Bs)-np.min(Bs))*0.01
        imax = isominmax[1]*(np.max(Bs)-np.min(Bs))*0.01

        t = figmag.data[4]
        t.colorbar.title = 'magB [mT]'
        t.update(x=x, y=y, z=z, value=Bs, visible=True, surface_count=surface_count, opacity = opacity, isomin=imin, isomax=imax)
        for v in isosurface_widgets_dict.values():
            v.disabled = False

    hide_button = widgets.Button(description='remove', button_style='warning', layout=dict(width='auto'))
    hide_button.on_click(lambda b: _clear_isosurface_data())
    iw = widgets.interactive(update_isosurface, {"manual":True, "manual_name":'update'}, **isosurface_widgets_dict)
    iw.manual_button.button_style='success'
    iw.manual_button.layout.width='auto'
    iwd = isosurface_widgets_dict
    isosurface_widgets = widgets.VBox([iwd['srx'], iwd['sry'], iwd['srz'], iwd['isominmax'], iwd['opacity'],
                                       widgets.HBox([widgets.HTML('density: '), 
                                                     iwd['density_x'], iwd['density_y'], iwd['density_z'], 
                                                     iwd['surface_count'], iw.manual_button, hide_button])
                                      ])
    return isosurface_widgets_dict, isosurface_widgets

# Update source

In [ ]:
@debug_view.capture(clear_output=True)
def update_magnet(source_id, angle, axis_x, axis_y, axis_z, xpos, ypos,zpos, Lx, Ly, Lz, Mx, My, Mz):
    global sources
    for k in ['x','y','z']:
        r = isosurface_options['density_'+k]
    _clear_isosurface_data()
    axis = (axis_x, axis_y, axis_z)
    sp = sources[source_id]
    if angle!=0:
        sp['magpy_source'].setOrientation(angle, axis)
    sp['magpy_source'].setPosition([xpos, ypos, zpos])
    sp['magpy_source'].magnetization = np.array([Mx,My,Mz])
    if sp['source_type'] == 'box':
        dim = sp['magpy_source'].dimension = np.array([Lx,Ly,Lz])
    elif sp['source_type'] == 'cylinder':
        dim = sp['magpy_source'].dimension = np.array([Lx,Ly])
    elif sp['source_type'] == 'sphere':
        dim = sp['magpy_source'].dimension = Lx
    tm = sp['trace']
    
    show_streamlines()
        
    with figmag.batch_update():
        tm.update(getSourceTrace(sp['magpy_source']))
            
@debug_view.capture(clear_output=True)
def update_current(source_id):
    global sources
    _clear_isosurface_data()
    sp = sources[source_id]
    w =sp['widgets']
    axis = (w['axis_x'].value, w['axis_y'].value, w['axis_z'].value)
    pos = (w['xpos'].value, w['ypos'].value, w['zpos'].value)
    angle = w['angle'].value
    curr = w['curr'].value
    
    if angle!=0:
        sp['magpy_source'].setOrientation(angle, axis)
    sp['magpy_source'].setPosition(pos)
    tm = sp['trace']
    
    show_streamlines()
        
    with figmag.batch_update():
        if sp['source_type'] == 'lineCurrent':
            vertices = sp['magpy_source'].vertices = [tuple(w[f'{n}{i}'].value for n in ['x','y','z']) for i,v in enumerate(sp['vertices'])]
            tm.update(getSourceTrace(sp['magpy_source']))
        elif sp['source_type'] == 'circularCurrent':
            dim = sp['magpy_source'].dimension = w['d'].value
            tm.update(getSourceTrace(sp['magpy_source']))
            
@debug_view.capture(clear_output=True)
def update_dipole(source_id):
    global sources
    _clear_isosurface_data()
    sp = sources[source_id]
    w =sp['widgets']
    axis = (w['axis_x'].value, w['axis_y'].value, w['axis_z'].value)
    pos = (w['xpos'].value, w['ypos'].value, w['zpos'].value)
    angle = w['angle'].value
    moment = (w['moment_x'].value, w['moment_y'].value, w['moment_z'].value)
    
    if angle!=0:
        sp['magpy_source'].setOrientation(angle, axis)
    sp['magpy_source'].setPosition(pos)
    sp['magpy_source'].moment = moment
    tm = sp['trace']
    
    show_streamlines()
        
    with figmag.batch_update():
        tm.update(getSourceTrace(sp['magpy_source'], dipolesizeref=w['sizeref'].value))

        
@debug_view.capture(clear_output=True)
def on_continuous_update_change(change):
    for s in sources.values():
        for v in s['widgets'].values():
            v.continuous_update = change.owner.value
        
        
@debug_view.capture(clear_output=True)
def update_source_title(change):
    sources_list_widget.set_title(sources_list_widget.selected_index, f"{change.owner.id}" if change.new.strip()=='' else f"{change.new}")


# Delete magnet

In [ ]:
@debug_view.capture(clear_output=True)
def on_delete_source_button_clicked(b):
    global sources
    pmc.removeSource(sources[b.id]['magpy_source'])
    sources.pop(b.id, sources)
    figmag.data = [figmag.data[i] for i in range(5)] + [v['trace'] for k,v in sources.items()]
    sources_list_widget.children = [v['widget'] for k,v in sources.items()]
    for i,v in enumerate([v for v in sources.values()]):
        sources_list_widget.set_title(i, f"{v['id']}" if v['name_widget'].value.strip()=='' else f"{v['name_widget'].value.strip()}") 
    if len(sources)==0:
        for v in streamlines_options.values():
            v['checkbox'].value= False
        graphics_container.children = [graphics_accordion]
    _clear_isosurface_data()
    show_streamlines()
        
@debug_view.capture(clear_output=True)
def delete_all_sources():
    global sources
    _clear_isosurface_data()
    figmag.data = figmag.data[:5]
    sources.clear()
    for m in sources.values():
        pmc.removeSource(m['magpy_source'])
    sources_list_widget.children=[]
    for v in streamlines_options.values():
        v['checkbox'].value= False
    graphics_container.children = [graphics_accordion]
    
def _clear_isosurface_data():
    figmag.data[4].x = []
    figmag.data[4].y = []
    figmag.data[4].z = []
    figmag.data[4].value = []

# Add source

In [ ]:
@debug_view.capture(clear_output=True)
def add_source(source_type='box', mag=(0,0,100),dim=(50,50,50), pos=(0,0,0), angle=0, axis=(0,0,1), curr=1, 
               vertices=[(-10,0,0),(10,0,0)], moment=(1,0,0), dipolesizeref=10, name=None):
    global all_widgets
    _clear_isosurface_data()
    graphics_container.children = [graphics_accordion, streamlines_accordion, isosurface_accordion]
    
    mag_props  = {'mag':mag, 'pos':pos,  'dim':dim, 'angle':angle, 'axis':axis}
    for i in range(1,100):
        source_id = f'{source_type}_{i:02d}'
        if source_id not in sources.keys():
            break
            
    delete_source_button = widgets.Button(description='delete', icon='trash', button_style='danger',
                                         layout=dict(width='auto'))
    delete_source_button.on_click(on_delete_source_button_clicked)
    delete_source_button.id = source_id
    source_name_widget = widgets.Text(description='name', value = '' if name == None else name)
    source_name_widget.id = source_id
    source_name_widget.observe(update_source_title, names = 'value')
    cst = 0.1 #color scale threshold
    if source_type=='box':
        magpy_source = magpy.source.magnet.Box(**mag_props)
        figmag.add_trace(getSourceTrace(source=magpy_source))
        dimensions_widgets = dict(Lx = widgets.FloatSlider(description='x [mm]', min=0, max=dim[0]*5, step=0.1, value=dim[0]),
                                Ly = widgets.FloatSlider(description='y [mm]', min=0, max=dim[1]*5, step=0.1, value=dim[1]),
                                Lz = widgets.FloatSlider(description='z [mm]', min=0, max=dim[2]*5, step=0.1, value=dim[2])
                                 )
    elif source_type=='cylinder':
        if len(dim)==2:
            dim = list(dim[0:2]) + [0]
        mag_props['dim'] = dim[0:2]
        magpy_source = magpy.source.magnet.Cylinder(**mag_props)
        figmag.add_trace(getSourceTrace(source=magpy_source))
        dimensions_widgets = dict(Lx = widgets.FloatSlider(description='d_outer [mm]', min=0, max=dim[0]*5, step=0.1, value=dim[0]),
                                Ly = widgets.FloatSlider(description='h [mm]', min=0, max=dim[1]*5, step=0.1, value=dim[1]),
                                Lz = widgets.FloatSlider(description='d_inner [mm]', min=0, max=dim[0]*5, step=0.1, value=dim[2])
                                 )
    elif source_type=='sphere':
        magpy_source = magpy.source.magnet.Sphere(**mag_props)
        figmag.add_trace(getSourceTrace(source=magpy_source))
        dimensions_widgets = dict(Lx = widgets.FloatSlider(description='r [mm]', min=0, max=dim*5, step=0.1, value=dim),
                                 Ly = widgets.FloatSlider(layout=dict(visibility='hidden')),
                                 Lz = widgets.FloatSlider(layout=dict(visibility='hidden'))
                                )

    orientation_widgets = dict(angle = widgets.FloatSlider(description='angle [deg]', min=-180, max=180, value=angle, 
                                                           style=dict(handle_color='blue')),
                               axis_x= widgets.FloatSlider(description='x', min=-1, max=1, step=0.1, value=axis[0]),
                               axis_y= widgets.FloatSlider(description='y', min=-1, max=1, step=0.1, value=axis[1]),
                               axis_z= widgets.FloatSlider(description='z', min=-1, max=1, step=0.1, value=axis[2])
                              )
    position_widgets = dict(xpos = widgets.FloatSlider(description='x [mm]', min=pos[0]-100, max=pos[0]+100, step=0.1, value=pos[0]),
                            ypos = widgets.FloatSlider(description='y [mm]', min=pos[1]-100, max=pos[1]+100, step=0.1, value=pos[1]),
                            zpos = widgets.FloatSlider(description='z [mm]', min=pos[2]-100, max=pos[2]+100, step=0.1, value=pos[2])
                           )
    
    magnetization_widgets = dict(Mx = widgets.FloatSlider(description='x [mT]', min=-abs(mag[0])*10-1, max=abs(mag[0])*10+1, step=0.1, value=mag[0]),
                                     My = widgets.FloatSlider(description='y [mT]', min=-abs(mag[1])*10-1, max=abs(mag[1])*10+1, step=0.1, value=mag[1]),
                                     Mz = widgets.FloatSlider(description='z [mT]', min=-abs(mag[2])*10-1, max=abs(mag[2])*10+1, step=0.1, value=mag[2])
                                    )
    
    all_widgets = dict(**orientation_widgets, **position_widgets)
        
    orient_buttons={}
    def _on_orient_button_click(b):
        for k,v in orientation_widgets.items():
            v.value = 1 if k == 'axis_' + b.description else 0
    for o in ['x','y','z']:
        ob = widgets.Button(description=o, icon='check', layout=dict(width='auto'))
        ob.on_click(_on_orient_button_click)
        orient_buttons[f'axis_{o}'] = ob
    orient_button_HBox = widgets.HBox(list(orient_buttons.values()), layout=dict(justify_content='space-between'))
    
    tabs = widgets.Tab([widgets.VBox(list(position_widgets.values()) + [continuous_update_checkbox]),
                        widgets.VBox(list(orientation_widgets.values()) + [orient_button_HBox] + [continuous_update_checkbox])
                       ])
    
    tabs.set_title(0,'position')
    tabs.set_title(1,'orientation')
       
    if source_type in ['box', 'sphere', 'cylinder']:
        all_widgets.update(**magnetization_widgets, **dimensions_widgets)
        widgets.interactive(update_magnet, source_id= widgets.Label(source_id), **all_widgets)
        tabs.children += (widgets.VBox(list(dimensions_widgets.values()) + [continuous_update_checkbox]), 
                        widgets.VBox(list(magnetization_widgets.values()) + [continuous_update_checkbox])
                       )
        tabs.set_title(2,'dimensions')
        tabs.set_title(3,'magnetization')
    
    elif source_type in ['lineCurrent', 'circularCurrent']:
        C = dict(curr = widgets.FloatSlider(description='current [A]', min=-10*curr, max=10*curr, step=0.1, value=curr))
        all_widgets.update(**C)
        if source_type == 'lineCurrent':
            magpy_source = magpy.source.current.Line(curr, vertices, pos, angle, axis)
            figmag.add_trace(getSourceTrace(source=magpy_source))
            v_dict = dict()
            for i,ver in enumerate(vertices):
                for k,v in zip([f'{n}{i}' for n in ['x','y','z']], ver):
                        v_dict[k] = widgets.FloatSlider(description=f'{k} [mm]', min=-100, max=100, step=0.1, value=v)
                        v_dict[k].id = source_id
            all_widgets.update(v_dict)
            for  v in all_widgets.values():
                v.observe(lambda change: update_current(source_id), names='value')
            tabs.children += (widgets.VBox(list(v_dict.values()) + [continuous_update_checkbox]),)
            tabs.set_title(2,'vertices')
        else:
            magpy_source = magpy.source.current.Circular(curr, dim, pos, angle, axis)
            figmag.add_trace(getSourceTrace(source=magpy_source))
            D = dict(d = widgets.FloatSlider(description='d [mm]', min=-10*dim, max=10*dim, step=0.1, value=dim))
            all_widgets.update(**D)
            for  v in all_widgets.Lvalues():
                v.observe(lambda change: update_current(source_id), names='value')
            tabs.children += (widgets.VBox(list(D.values()) + [continuous_update_checkbox]),)
            tabs.set_title(2,'dimension')
        
        tabs.children += (widgets.VBox(list(C.values()) + [continuous_update_checkbox]),)
        tabs.set_title(3,'current')
    elif source_type == 'dipole':  
        magpy_source = magpy.source.moment.Dipole(moment=moment, pos=pos, angle=angle, axis=axis)
        figmag.add_trace(getSourceTrace(source=magpy_source, dipolesizeref=dipolesizeref))
        M = dict(moment_x = widgets.FloatSlider(description='x [mT*mm^3]', min=-10, max=10, step=0.1, value=moment[0]),
                 moment_y = widgets.FloatSlider(description='y [mT*mm^3]', min=-10, max=10, step=0.1, value=moment[1]),
                 moment_z = widgets.FloatSlider(description='z [mT*mm^3]', min=-10, max=10, step=0.1, value=moment[2]),
                 sizeref = widgets.FloatSlider(description='sizeref', min=0, max=100, step=0.1, value=dipolesizeref),
                )
        all_widgets.update(**M)
        for  v in all_widgets.values():
            v.observe(lambda change: update_dipole(source_id), names='value')
        tabs.children += (widgets.VBox(list(M.values()) + [continuous_update_checkbox]),)
        tabs.set_title(2,'moment')
            
    for w in all_widgets.values():
        w.style.description_width='auto'
        w.layout.width='auto'
    
    pmc.addSources(magpy_source)    
    
    def rename_source(b):
        if b.icon=='check':
            source_name_container.children =[rename_source_button, source_opacity_slider]
        else:
            source_name_container.children =[source_name_widget, ok_button]
    
    rename_source_button = widgets.Button(description='rename', icon='text', button_style='warning',layout=dict(width='auto'))
    rename_source_button.on_click(rename_source)
    ok_button = widgets.Button(icon='check', button_style='success',layout=dict(width='auto'))
    ok_button.on_click(rename_source)
    source_opacity_slider = widgets.FloatSlider(description='opacity', value=1, min=0, max=1)
    widgets.jsdlink((opacity_slider, 'value'), (source_opacity_slider, 'value'))
    def update_source_opacity(opacity):
        figmag.data[-1].opacity = opacity
    widgets.interactive(update_source_opacity, opacity=source_opacity_slider)
    
    source_name_container = widgets.HBox([rename_source_button, source_opacity_slider])
    sources[source_id] = {'id': source_id, 'source_type':source_type, 'name_widget':source_name_widget, 'trace':figmag.data[-1], 
                          'properties':mag_props, 'magpy_source': magpy_source, 'widgets': all_widgets,
                          'widget': widgets.VBox([tabs, widgets.HBox([source_name_container , delete_source_button], 
                                                                     layout=dict(justify_content='space-between'))])}
    if source_type=='lineCurrent':
        sources[source_id].update(dict(vertices=vertices))
    sources_list_widget.children+=(sources[source_id]['widget'],)
    sources_list_widget.set_title(len(sources_list_widget.children)-1, f"{source_id}" if name==None else f"{name}")
    sources_list_widget.selected_index = len(sources_list_widget.children)-1
    

#  Json handling 

In [ ]:
@debug_view.capture(clear_output=True)
def get_dict():
    output_dict = {'sources': {}, 'sensors':{}, 'layout': {}}
    so = output_dict['sources']
    for k,v in sources.items():
        so[k]={'name':v['name_widget'].value,
              'id': v['id'],
              'source_type': v['source_type']}
        props = {}
        for k1,v1 in v['properties'].items():
            if isinstance(v, np.ndarray):
                props[k1] = v1.tolist()
            else:
                props[k1] = v1
        so[k]['properties'] = props
    return output_dict

@debug_view.capture(clear_output=True)
def load_dict(input_dict):
    delete_all_sources()
    for source in input_dict['sources'].values():
        add_source(source_type = source['source_type'], **source['properties'])

@debug_view.capture(clear_output=True)
def on_json_file_selector_change(filepath=None):
    if filepath.strip() != '':
        filepath = os.path.join(json_folder,f'{filepath.strip()}.json')
        if os.path.isfile(filepath):
            delete_state_button.disabled = False
            save_state_button.disabled = True
            load_json_button.disabled = False
        else:
            delete_state_button.disabled = True
            save_state_button.disabled = False
            load_json_button.disabled = True
    else:
        delete_state_button.disabled = True
        save_state_button.disabled = False
        load_json_button.disabled = True


@debug_view.capture(clear_output=True)
def load_json(filepath):
    with open(filepath, 'r') as fp:
        p_input = json.load(fp)
    load_dict(input_dict = p_input)
            
@debug_view.capture(clear_output=True)
def save_json(mydict, filename='input_data.json', folder='saved_sources_sets'):
    if not os.path.isdir(folder):
        os.mkdir(folder)
    with open(os.path.join(folder,filename), 'w') as fp:
        json.dump(mydict, fp, sort_keys=True, indent=4)


@debug_view.capture(clear_output=True)
def _on_save_state_button_click(b=None):
    fns =  json_file_selector.value
    mydict=  get_dict()
    for i in range(1,100):
        fn = f'{fns}.json'
        if not os.path.isfile(os.path.join(json_folder,fn)):
            break
    save_json(mydict, fn, json_folder)
    update_json_selector_options()
    delete_state_button.disabled = False
    save_state_button.disabled = True

@debug_view.capture(clear_output=True)
def update_json_selector_options(reset=False):
    json_paths = [file for file in glob.glob(os.path.join(json_folder, '*.json'))]
    json_filenames = [os.path.basename(f) for f in  json_paths]
    json_file_selector.options = [os.path.splitext(fn)[0] for fn in  json_filenames]
    if reset:
        json_file_selector.value = ''
        
@debug_view.capture(clear_output=True)
def _on_reset_state_button_click(b):
    json_file_selector.index = 0
    json_file_selector.value = ''
    update_json_selector_options(reset=True)

@debug_view.capture(clear_output=True)
def _on_delete_button_click(b):
    filepath =   os.path.join(json_folder, f'{json_file_selector.value.strip()}.json')
    os.remove(filepath)
    update_json_selector_options(reset=False)

In [ ]:
json_folder = 'saved_sources_sets'

save_state_button = widgets.Button(tooltip='save current state to json file', button_style='info', icon = 'save', disabled=True, 
                                        layout=dict(width='auto'))
save_state_button.on_click( _on_save_state_button_click)
reset_state_button = widgets.Button(tooltip='reset and resfresh textbox options', button_style='warning', icon = 'repeat', 
                                         layout=dict(width='auto'))
reset_state_button.on_click( _on_reset_state_button_click)
delete_state_button = widgets.Button(tooltip='delete current state json file', disabled=True, button_style='danger', icon = 'trash', 
                                          layout=dict(width='auto'))
delete_state_button.on_click( _on_delete_button_click)

load_json_button = widgets.Button(tooltip='load state', button_style='success', icon = 'upload', disabled=True, 
                                        layout=dict(width='auto'))
load_json_button.on_click(lambda b: load_json(os.path.join(json_folder,f'{json_file_selector.value.strip()}.json')))
json_file_selector = widgets.Combobox(placeholder='enter name or choose from list')
json_file_selector.style.description_width = '0px'
json_file_selector.layout.width = '350px'
update_json_selector_options(reset=True)
widgets.interactive(on_json_file_selector_change, filepath = json_file_selector)

json_widgets = widgets.HBox([])

json_widgets.children = [json_file_selector, load_json_button,  reset_state_button,  save_state_button,  delete_state_button]

# Definitions

In [ ]:
pmc = magpy.Collection()

figmag = go.FigureWidget()
figmag.add_scatter3d(name='ref coord-sys', mode='lines+text', 
                     x=[0,10,0,0,0,0], y=[0,0,0,10,0,0], z=[0,0,0,0,0,10], 
                     text=['','x','','y','','z'], textfont_color = 'blue')
for k in ['xy', 'xz', 'yz']:
    figmag.add_trace(dict(type='scatter3d', name = f'streamline {k}'))
figmag.add_isosurface()
scene_range = 100
figmag.layout.scene.aspectmode = 'auto'
figmag.layout.scene = dict(xaxis_title = 'x[mm]', yaxis_title = 'y[mm]', zaxis_title = 'z[mm]')
figmag.layout.margin=dict(l=0,r=0,t=10,b=10)
figmag.layout.showlegend = True
figmag.layout.legend.orientation = 'h'

sources={}
sensors={}
add_box_button = widgets.Button(description='box', icon='plus', layout=dict(flex='1'), style=dict(button_color='#E74C3C'))
add_box_button.on_click(lambda b: add_source(source_type='box'))
add_cylinder_button = widgets.Button(description='cylinder', icon='plus', layout=dict(flex='1'), style=dict(button_color='#8E44AD'))
add_cylinder_button.on_click(lambda b: add_source(source_type='cylinder', dim=(60,60)))
add_sphere_button = widgets.Button(description='sphere', icon='plus', layout=dict(flex='1'), style=dict(button_color='#3498DB'))
add_sphere_button.on_click(lambda b: add_source(source_type='sphere', dim=70))
add_dipole_button = widgets.Button(description='dipole', icon='plus', layout=dict(flex='1'), style=dict(button_color='#2ECC71'))
add_dipole_button.on_click(lambda b: add_source(source_type='dipole', moment=(0,0,10)))
add_line_button = widgets.Button(description='line current', icon='plus', layout=dict(flex='1'), style=dict(button_color='#F1C40F'))
add_line_button.on_click(lambda b: add_source(source_type='lineCurrent', vertices=[(-100,0,0),(100,0,0)]))
add_circle_button = widgets.Button(description='circlular current', icon='plus', layout=dict(flex='1'), style=dict(button_color='#E67E22'))
add_circle_button.on_click(lambda b: add_source(source_type='circularCurrent', dim=70))
add_source_buttons = widgets.VBox([widgets.HBox([add_box_button, add_cylinder_button, add_sphere_button]),
                                   widgets.HBox([add_dipole_button, add_line_button, add_circle_button])
                                  ])
continuous_update_checkbox = widgets.Checkbox(description = 'continuous update', value=True)
continuous_update_checkbox.observe(on_continuous_update_change, names='value')

        
streamlines_options, streamline_widgets = _define_streamlines_widgets()
streamlines_accordion = widgets.Accordion([streamline_widgets])
streamlines_accordion.set_title(0,'Streamlines')
streamlines_accordion.selected_index = None

isosurface_options, isosurface_widgets = _define_isosurface_widgets()
isosurface_accordion = widgets.Accordion([isosurface_widgets])
isosurface_accordion.set_title(0,'Isosurface')
isosurface_accordion.selected_index = None


def set_opacity(opacity):
    for m in sources.values():
        m['trace'].opacity = opacity
opacity_slider =  widgets.FloatSlider(description='sources opacity', min=0, max=1, step=0.1, value=1, style=dict(description_width='auto'))
widgets.interactive(set_opacity, opacity =opacity_slider)

def set_fig_template(template):
    figmag.layout.template = template
fig_template_dropdown = widgets.Dropdown(description='theme', value = default_theme, 
                                         options=['plotly_grey', 'plotly', 'plotly_white'], 
                                         style=dict(description_width='auto'), layout=dict(width='150px'))
widgets.interactive(set_fig_template, template=fig_template_dropdown)


auto_scene_range_checkbox = widgets.ToggleButton(value=True, icon='sliders', description='modify scene range', layout=dict(width='auto'), style=dict(description_width='0px'))
scene_range_dict = dict(
    xmin = widgets.FloatText(description='x-min', value=-100, layout=dict(width='100px'), style=dict(description_width='auto')),
    xmax = widgets.FloatText(description='x-max', value=100, layout=dict(width='100px'), style=dict(description_width='auto')),
    ymin = widgets.FloatText(description='y-min', value=-100, layout=dict(width='100px'), style=dict(description_width='auto')),
    ymax = widgets.FloatText(description='y-max', value=100, layout=dict(width='100px'), style=dict(description_width='auto')),
    zmin = widgets.FloatText(description='z-min', value=-100, layout=dict(width='100px'), style=dict(description_width='auto')),
    zmax = widgets.FloatText(description='z-max', value=100, layout=dict(width='100px'), style=dict(description_width='auto')),
)
scene_range_widgets = widgets.HBox(layout=dict(flex_flow='wrap'))

@debug_view.capture(clear_output=True)
def update_scene_range(auto, xmin,xmax, ymin, ymax, zmin, zmax):
    sc = figmag.layout.scene
    x, y, z = [xmin, xmax] , [ymin, ymax] , [zmin, zmax]
    if not auto:
        auto_scene_range_checkbox.description = 'auto range'
        scene_range_widgets.children = list(scene_range_dict.values())
        with figmag.batch_animate():
            sc.xaxis.autorange = False
            sc.yaxis.autorange = False
            sc.zaxis.autorange = False
            sc.xaxis.range = x
            sc.yaxis.range = y 
            sc.zaxis.range = z
            figmag.layout.scene.aspectmode = 'auto'
    else:
        auto_scene_range_checkbox.description = 'modify scene range'
        scene_range_widgets.children = []
        with figmag.batch_animate():
            sc.xaxis.autorange = True
            sc.yaxis.autorange = True
            sc.zaxis.autorange = True
            figmag.layout.scene.aspectmode = 'auto'
    
widgets.interactive(update_scene_range, auto = auto_scene_range_checkbox, **scene_range_dict)
scene_range_container = widgets.HBox([auto_scene_range_checkbox, scene_range_widgets])


@debug_view.capture(clear_output=True)
def f(obj,x,y,z):
    for k,r in zip(['x','y','z'], [x,y,z]):
        sr = isosurface_options['sr'+k]
        sr.min, sr.max = r
        sr.value = r
figmag.layout.scene.on_change(f, ('xaxis', 'range'), ('yaxis', 'range'), ('zaxis', 'range'))

sources_list_widget = widgets.Accordion([])
sources_window = widgets.Accordion([widgets.VBox([json_widgets, add_source_buttons, sources_list_widget])])
sources_window.set_title(0,'Sources')
graphics_accordion = widgets.Accordion([widgets.VBox([figmag, 
                                                      widgets.HBox([opacity_slider, fig_template_dropdown, scene_range_container],
                                                                  layout=dict(flex_flow='wrap'))],
                                                     layout=dict(flex_flow='wrap', max_width='98%'))])
graphics_accordion.set_title(0, 'Graphics')
graphics_container = widgets.VBox([graphics_accordion])
graphics_container.layout.flex='3'
graphics_container.layout.min_width='500px'
sources_window.layout.flex='1'
gui = widgets.VBox([widgets.HBox([sources_window, graphics_container], layout=dict(flex_flow='wrap')),debug_view])

# Testing

In [ ]:
display(gui)

In [ ]:
class UnboudedFloatSlider(widgets.HBox):
    def __init__(self, description='', min=-1, max=1, value=0, **kwargs):
        widgets.HBox.__init__(self, **kwargs)
        widgets.IntSlider.__init__(self, value=value, description=description)
        s = widgets.IntSlider(min=min, max=max, value=value, readout=True,layout=dict(flex='1'))
        sval = widgets.IntText(description=description, min=min, max=max, value=value, style=dict(description_width='auto'), layout=dict(width='auto'))
        smin = widgets.IntText(description='min', style=dict(description_width='auto'), layout=dict(width='85px'))
        smax = widgets.IntText(description='max', style=dict(description_width='auto'), layout=dict(width='85px'))
        widgets.jslink((s,'max'), (smax,'value'))
        widgets.jslink((s,'min'), (smin,'value'))
        widgets.jslink((s,'value'), (sval,'value'))
        def on_val_change(change):
            if change.new> smax.value:
                s.min = change.new
            if change.new< smin.value:
                s.min = change.new
            if change.new!=change.old:
                s.value = change.new
        sval.observe(on_val_change, names='value')
        self.sval = sval
        self.children = [sval, smin,s, smax]
        self.description = description
        self.style = sval.style
        
    @property
    def value(self):
        return self.sval.value

    @value.setter
    def value(self, newvalue):
        self.sval.value  = newvalue